In [1]:
import pyspark
from delta import *
from confluent_kafka.schema_registry import SchemaRegistryClient
from pyspark.sql.avro.functions import from_avro
from pyspark.sql.functions import expr

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.12:3.0.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1,org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.1,io.delta:delta-core_2.12:0.8.0,com.amazonaws:aws-java-sdk:1.11.950,org.apache.hadoop:hadoop-aws:3.2.0,net.java.dev.jets3t:jets3t:0.9.4 pyspark-shell'

builder = pyspark.sql.SparkSession.builder \
    .appName("kafka-delta-app") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("fs.s3a.path.style.access", True) \
    .config("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("com.amazonaws.services.s3.enableV4", True) \
    .config("spark.driver.extraJavaOptions", "-Dcom.amazonaws.services.s3.enableV4=true") \
    .config("fs.s3a.access.key", "crypto-prices") \
    .config("fs.s3a.secret.key", "crypto-prices") \
    .config("fs.s3a.endpoint", "http://minio:9000")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

spark.conf.set("spark.sql.adaptive.enabled", False)

In [3]:
schema_registry_conf = {'url': "http://schema-registry:8081"}

schema_registry_client = SchemaRegistryClient(schema_registry_conf)
schema_response = schema_registry_client.get_latest_version("crypto-prices" + "-value").schema
schema = schema_response.schema_str

kafka_topic_name = "crypto-prices"
kafka_bootstrap_servers = 'kafka:9092'

dsraw = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
        .option("subscribe", kafka_topic_name) \
        .option("startingOffsets", "earliest") \
        .option("failOnDataLoss", "false") \
        .load()

from_avro_options= {"mode":"PERMISSIVE"}

ds = (
  dsraw
  .select(from_avro(expr("substring(value, 6, length(value)-5)"), schema, from_avro_options).alias("value"))
  .selectExpr(
      "value.id", 
      "value.rank", 
      "value.symbol", 
      "value.name", 
      "value.supply", 
      "value.maxSupply", 
      "value.marketCapUsd", 
      "value.volumeUsd24Hr", 
      "value.priceUsd", 
      "value.changePercent24Hr", 
      "value.vwap24Hr", 
      "value.explorer") \
)

delta_table = ds.writeStream \
  .format("delta") \
  .outputMode("append") \
  .option("checkpointLocation", "/delta/events/_checkpoints/etl-from-json") \
  .start("s3a://crypto-prices/events")

In [13]:
crypto_prices = spark.read.format("delta").load("s3a://crypto-prices/events")
crypto_prices.count()

13500

In [12]:
# Leer y ver en consola en real time el streaming
# stream2 = spark.readStream.format("delta").load("events").writeStream.format("console").start()